In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('titanic.csv')

In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
#checking for any missing values
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
x = data[['Pclass','SibSp','Parch','Fare']]
y = data['Survived']

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=42)

In [10]:
#scaling data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)   
x_test = sc.transform(x_test)

# **SVM**

In [11]:
from sklearn.svm import SVC
svcmodel = SVC(gamma='auto')
svcmodel.fit(x_train,y_train)
y_predsvm = svcmodel.predict(x_test)
accuracy_score(y_predsvm,y_test)

0.7309417040358744

5 fold cross validation

In [12]:
from sklearn.model_selection import cross_val_score,KFold
kfold = KFold(n_splits=5)
score = cross_val_score(SVC(gamma='auto'),x,y,cv=kfold,verbose=2)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
Cross Validation Scores are [0.6424581  0.65168539 0.71910112 0.69662921 0.79213483]
Average Cross Validation score :0.7004017324712825


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.5s finished


maximum accuracy obtained is 0.79213483

# **Random search**

In [13]:
parameters ={  'C': [0.1, 1, 10, 100, 1000],
                'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                'kernel': ['rbf']
            }

           

In [14]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,
                            n_iter=50)
random_result = random_search.fit(x_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 25 is smaller than n_iter=50. Running 25 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


Best Score:  71.7080013466502
Best Params:  {'kernel': 'rbf', 'gamma': 0.01, 'C': 100}


# **Hyper opt**

In [15]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [23]:
space ={  'C': [0.1, 1, 10, 100, 1000],
          'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
          'kernel': ['rbf']
            }

In [32]:
# define objective function

def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, x, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [48]:
space = {
      'C': hp.choice('C',[0.1, 1, 10, 100, 1000]),
      'kernel': hp.choice('kernel',['rbf']),
      'gamma':hp.choice('gamma',[1, 0.1, 0.01, 0.001, 0.0001])
      }

In [49]:
space

{'C': <hyperopt.pyll.base.Apply at 0x7fc6bf4f0b90>,
 'kernel': <hyperopt.pyll.base.Apply at 0x7fc6bf4f0690>,
 'gamma': <hyperopt.pyll.base.Apply at 0x7fc6bf4f0dd0>}

In [50]:
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
   space = space, 
        algo = tpe.suggest, 
            max_evals = 2,
            trials = trials
)

print("Best: {}".format(best))

100%|██████████| 2/2 [00:01<00:00,  1.95it/s, best loss: -0.6869374176134581]
Best: {'C': 4, 'gamma': 3, 'kernel': 0}
